In [79]:
import os
import sys
import numpy as np
import argparse
from astropy.table import Table

user = os.environ['USER']
sys.path.append('/cosma/home/durham/{}/DESI'.format(user))

from   delta8_limits import delta8_tier, d8_limits

#parser = argparse.ArgumentParser(description='Generate Gold luminosity function.')
#parser.add_argument('-f', '--field', type=str, help='Select equatorial GAMA field: G9, G12, G15', required=False, default=None)
    
#args   = parser.parse_args()
#field = args.field

# TODO: Add to headers.
tmr_DDP1     = [-21.8, -20.1]
tmr_DDP2     = [-20.6, -19.3]
tmr_DDP3     = [-19.6, -17.8]

Mr = np.array([tmr_DDP1, tmr_DDP2, tmr_DDP3])


'''
START HERE
'''

field = 'G9'
rpath = '/cosma/home/durham/{}/data/GAMA4/randoms/randoms_bd_ddp_n8_{}_0.fits'.format(user, field)
rand_G9 = Table.read(rpath)

field = 'G12'
rpath = '/cosma/home/durham/{}/data/GAMA4/randoms/randoms_bd_ddp_n8_{}_0.fits'.format(user, field)
rand_G12 = Table.read(rpath)

field = 'G15'
rpath = '/cosma/home/durham/{}/data/GAMA4/randoms/randoms_bd_ddp_n8_{}_0.fits'.format(user, field)
rand_G15 = Table.read(rpath)


abs_mag = []
ddp_min = []
ddp_max = []
ddp_ngal = []
ddp_vol = []
ddp_dens = []
ddp = [1,2,3]

for idx in ddp:
    abs_mag.append(Mr[idx-1])
    ddp_min.append(round(rand.meta['DDP{}_ZMIN'.format(idx)], 3))
    ddp_max.append(round(rand.meta['DDP{}_ZMAX'.format(idx)], 3))
    ddp_ngal.append(rand.meta['DDP{}_NGAL'.format(idx)])
    ddp_vol.append(rand.meta['DDP{}_VZ'.format(idx)])
    ddp_dens.append(rand.meta['DDP{}_DENS'.format(idx)])

ddp_ngal = np.array(ddp_ngal) / 10**3
ddp_vol = np.array(ddp_vol) / 10**6
ddp_dens = np.array(ddp_dens) / 10**-3

ddp_ngal = np.round(ddp_ngal, 2)
ddp_vol = np.round(ddp_vol, 2)
ddp_dens = np.round(ddp_dens, 2)

# Generate Table 2 of McNaught-Roberts (2014)
t1 = Table([ddp, abs_mag, ddp_min, ddp_max, ddp_ngal, ddp_vol, ddp_dens], names=('DDP', 'M_r^e - 5\\log_{10}h', 'z_{\\rm min}', 'z_{\\rm max}', 'N_{DDP}/10^3','V_{DDP}/ (10^6 h^{-3} Mpc^3)', '\\rho_{DDP}/(10^{-3} h^3 {\\rm Mpc}^{-3})'))

fscale = []
n_ddp = []
label = ['d0', 'd1', 'd2', 'd3']

print('bin', 'overdensities', 'f_delta', 'n_gal/10^3')
for idx in range(4):
    
    fpath = '/cosma/home/durham/{}/data/GAMA4/gama_gold_{}_ddp_n8_d0_{}.fits'.format(user, field, idx)
    dat = Table.read(fpath)
    n_ddp.append(len(dat))
    fscale.append(rand.meta['DDP1_d{}_VOLFRAC'.format(idx)])

n_ddp = np.array(n_ddp) / 10**3 
fscale = np.array(fscale)

n_ddp = np.round(n_ddp, 3)
fscale = np.round(fscale, 3)


# Generate Table 3 of McNaught-Roberts (2014)
t2 = Table([label, d8_limits, fscale, n_ddp], names=('Label', '\\delta_8', 'f_{\\delta}', 'N_{\\delta, DDP1}/10^3'))
    
print(t1)
print(t2)

bin overdensities f_delta n_gal/10^3
DDP M_r^e - 5\log_{10}h [2] z_{\rm min} z_{\rm max} N_{DDP}/10^3 V_{DDP}/ (10^6 h^{-3} Mpc^3) \rho_{DDP}/(10^{-3} h^3 {\rm Mpc}^{-3})
--- ----------------------- ----------- ----------- ------------ ---------------------------- ---------------------------------------
  1          -21.8 .. -20.1       0.039       0.263        45.89                         7.59                                    6.04
  2          -20.6 .. -19.3       0.039       0.194         32.7                         3.16                                   10.36
  3          -19.6 .. -17.8       0.039       0.102         8.75                         0.46                                   19.11
Label  \delta_8 [2]  f_{\delta} N_{\delta, DDP1}/10^3
----- -------------- ---------- ---------------------
   d0  -1.0 .. -0.75      0.288                 1.238
   d1    -0.4 .. 0.0      0.163                 4.187
   d2     0.7 .. 1.6      0.103                 7.175
   d3 4.0 .. 10000.0   

In [80]:
ascii.write(t2, Writer=ascii.Latex, latexdict=ascii.latex.latexdicts['AA'])

\begin{table}
\begin{tabular}{cccc}
\hline \hline
Label & \delta_8 & f_{\delta} & N_{\delta, DDP1}/10^3 \\
\hline
d0 & -1.0 .. -0.75 & 0.288 & 1.238 \\
d1 & -0.4 .. 0.0 & 0.163 & 4.187 \\
d2 & 0.7 .. 1.6 & 0.103 & 7.175 \\
d3 & 4.0 .. 10000.0 & 0.008 & 5.087 \\
\hline
\end{tabular}
\end{table}


In [62]:
print(rand_G12.meta['DDP1_NGAL'],rand_G15.meta['DDP1_NGAL'])

45889 45889


In [63]:
print(rand_G12.meta['DDP1_VZ'],rand_G15.meta['DDP1_VZ'])

7594754.153764196 7594754.153764196


In [64]:
print(rand_G12.meta['DDP1_DENS'],rand_G15.meta['DDP1_DENS'])

0.006042196899455394 0.006042196899455394


In [65]:
print(rand_G12.meta['DDP1_d0_VOLFRAC'],rand_G15.meta['DDP1_d0_VOLFRAC'])

0.2577701234782471 0.2698145851273098


In [67]:
print(len(rand_G9), len(rand_G12), len(rand_G15))

1265955 1265955 1265955


In [70]:
len(rand_G9[rand_G9['IN_DDP1'] == True])

1265810

In [72]:
len(rand_G12[rand_G12['IN_DDP1'] == True])

1265810

In [71]:
len(rand_G15[rand_G15['IN_DDP1'] == True])

1265810

In [61]:
rand.meta

OrderedDict([('ZMIN', 0.039),
             ('ZMAX', 0.263),
             ('DZ', 0.0001),
             ('NRAND', 1265955),
             ('FIELD', 'G9'),
             ('AREA', 180.0),
             ('BOUND_PERCENT', 1.0),
             ('VOL', 2531908.950723488),
             ('RAND_DENS', 0.5),
             ('VOL8', 2144.660584850632),
             ('NRAND8', 1072.330292425316),
             ('NRAND8_PERR', 32.74645465428763),
             ('RSPHERE', 8.0),
             ('FILLFACTOR_INFRAC', 0.7773617545647357),
             ('EXTNAME', 'TilingCat'),
             ('TCOMM1', 'GAMA ID'),
             ('TUCD1', 'meta.id;meta.main'),
             ('TCOMM2', 'SDSS objid'),
             ('TUCD2', 'meta.id'),
             ('TCOMM3', 'J2000 coordinate'),
             ('TUCD3', 'pos.eq.ra'),
             ('TCOMM4', 'J2000 coordinate'),
             ('TUCD4', 'pos.eq.dec'),
             ('TCOMM5', '3-arcsec fiber magnitude'),
             ('TUCD5', 'phot.mag;em.opt.R'),
             ('TCOMM6', 'Ext